In [62]:
import numpy as np
import scipy.stats as ss
import datetime as dt

# Black Scholes option price
# Function approach

In [4]:
# Black-Scholes option price

def bsm_option_price(strike, spot, vol, texp, intr=0.0, divr=0.0, callput=1):
    
    if (callput > 0):
        cp_sign = 1.0 
    else:
        cp_sign = -1.0
    
    vol_std = vol * np.sqrt(texp)
    div_fac = np.exp(-texp*divr)
    disc_fac = np.exp(-texp*intr)
    forward = spot / disc_fac * div_fac
    d1 = np.log(forward/strike)/vol_std + 0.5*vol_std
    d2 = d1 - vol_std

    price = cp_sign * disc_fac \
        * ( forward * ss.norm.cdf(cp_sign*d1) - strike * ss.norm.cdf(cp_sign*d2) )
    return price

### Different ways of using function arguments

In [5]:
### full arguments
c1 = bsm_option_price(105, 100, 0.2, 0.25, 0, 0, 1)

### omit arguments with default vaules
c2 = bsm_option_price(105, 100, 0.2, 0.25)

### you need to pass all required arguments
#c2 = bsm_option_price(105, 100, 0.2)

### use argument names
c3 = bsm_option_price(105, 100, texp=0.25, vol=0.2)

### always put the named args at the end
#c3 = bsm_option_price(105, texp=0.25, 100, vol=0.2)

c1, c2, c3

(2.0640191378988462, 2.0640191378988462, 2.0640191378988462)

## Disadvantage of function approach:
* The computation in the function has to be repeated every time.
* Some part doesn't have to be repeated: for example, __vol, texp, intr, divr__ don't change often. So   __vol_std, disc_fac, div_fac__ don't have to be computed.
* You feel some needs to organize by binding some data + function(method)

# Class approach
We're going to bind __vol, int_r, int_d__ as a class

In [174]:
class BSM_Ver1:
    def __init__(self, vol, texp, intr=0.0, divr=0.0):
        ''' Constructor for this class. '''
        # Create some member animals
        self.vol, self.texp, self.intr, self.divr = vol, texp, intr, divr
    
    def price(self, strike, spot, cp_sign=1):
        # cp_sign = 1 for call, -1 for put
        vol_std = self.vol * np.sqrt(self.texp)
        div_fac = np.exp(-self.texp*self.divr)
        disc_fac = np.exp(-self.texp*self.intr)
        forward = spot / disc_fac * div_fac
        d1 = np.log(forward/strike)/vol_std + 0.5*vol_std
        d2 = d1 - vol_std

        price = cp_sign * disc_fac \
            * ( forward * ss.norm.cdf(cp_sign*d1) - strike * ss.norm.cdf(cp_sign*d2) )
        return price

In [175]:
### define a bsm model with spot, vol and expiry.
bsm1 = BSM_Ver1(vol=0.2, texp=1, intr=0.0)

print( bsm1.__dict__ )

### price options with strike
bsm1.price(102, 100), bsm1.price(strike=102, spot=100, cp_sign=-1)

{'texp': 1, 'vol': 0.2, 'intr': 0.0, 'divr': 0.0}


(7.0844942478298947, 9.0844942478298805)

In [176]:
### Lets change vol/expiry
bsm1.vol = 0.4
#bsm1.texp = 3

### option price changes
bsm1.price(102, 100), bsm1.price(strike=102, spot=100, cp_sign=-1)

(15.029816377425, 17.029816377425)

In [177]:
class BSM_Ver2:
    def __init__(self, vol, texp, intr=0.0, divr=0.0):
        ''' Constructor for this class. '''
        # Create some member animals
        self.vol, self.texp, self.intr, self.divr = vol, texp, intr, divr
        
        self.vol_std = vol * np.sqrt(texp)
        self.div_fac = np.exp(-texp*divr)
        self.disc_fac = np.exp(-texp*intr)
    
    def price(self, strike, spot, cp_sign=1):
        forward = spot / self.disc_fac * self.div_fac
        d1 = np.log(forward/strike)/self.vol_std + 0.5*self.vol_std
        d2 = d1 - self.vol_std
        price = cp_sign * self.disc_fac \
            * ( forward * ss.norm.cdf(cp_sign*d1) - strike * ss.norm.cdf(cp_sign*d2) )
        return price

In [178]:
bsm2 = BSM_Ver2(vol=0.2, texp=1)

bsm2.price(102, 100), bsm2.price(102, 100, cp_sign=-1)

(7.0844942478298947, 9.0844942478298805)

In [179]:
### But... now things are not that simple
bsm2.vol = 0.4
### option prices are price same after volatility change !@#$
bsm2.price(102, 100), bsm2.price(102, 100, cp_sign=-1)
#print( bsm2.__dict__ )

(7.0844942478298947, 9.0844942478298805)

In [185]:
class BSM_Ver3:
    def __init__(self, vol, texp, intr=0.0, divr=0.0):
        ''' Constructor for this class. '''
        # you need to 'initialize' all data members in __init__ function
        self.vol, self.texp, self.intr, self.divr = vol, texp, intr, divr
        self.setparams()

    def setparams(self, vol=None, texp=None, intr=None, divr=None):
        self.vol = vol or self.vol
        self.texp = texp or self.texp
        self.intr = intr or self.intr
        self.divr = divr or self.divr
        
        self.vol_std = self.vol * np.sqrt(self.texp)
        self.div_fac = np.exp(-self.texp*self.divr)
        self.disc_fac = np.exp(-self.texp*self.intr)
    
    def price(self, strike, spot, cp_sign=1):
        forward = spot / self.disc_fac * self.div_fac
        d1 = np.log(forward/strike)/self.vol_std + 0.5*self.vol_std
        d2 = d1 - self.vol_std
        price = cp_sign * self.disc_fac \
            * ( forward * ss.norm.cdf(cp_sign*d1) - strike * ss.norm.cdf(cp_sign*d2) )
        return price
    
    def delta(self, strike, spot, cp_sign=1):
        forward = spot / self.disc_fac * self.div_fac
        d1 = np.log(forward/strike)/self.vol_std + 0.5*self.vol_std
        delta = cp_sign * ss.norm.cdf(cp_sign*d1)
        return delta

In [186]:
bsm3 = BSM_Ver3(vol=0.2, texp=1)

bsm3.price(102, 100, 1), bsm3.price(102, 100, -1), bsm3.delta(102, 100)

(7.0844942478298947, 9.0844942478298805, 0.50039370151885099)

In [188]:
### not let's try setparams method
bsm3.setparams(vol=0.4, intr=0.01)
### option prices are price same after volatility change !@#$
bsm3.price(102, 100, 1), bsm3.price(102, 100, -1), bsm3.delta(102, 100)

#print( bsm2.__dict__ )

(15.442197598441437, 16.42728064085658, 0.56965403430446548)

In [214]:
class OptionContract:
    def __init__(self, undl, opt_type, strike, dexp, dtoday=dt.date.today()):
        ''' Constructor for this class. '''
        self.undl, self.strike, self.dexp, self.dtoday = undl, strike, dexp, dtoday
        self.opt_type = opt_type
        self.cp_sign = 1 if (opt_type == 'call') else -1
        self.texp = (dexp - dtoday).days/365.25

    def toString(self):
        return('{:s} option on {:s} struck at {:0.1f} maturing on {:s}'\
              .format(self.opt_type, self.undl, self.strike, self.dexp.strftime('%Y.%m.%d')))

    def price(self, spot, model):
        if(abs(model.texp - self.texp)>1e-12):
            print('Resetting texp of model ...')
            model.setparams(texp=self.texp)
        return model.price(self.strike, spot, cp_sign=self.cp_sign)
    
    def delta(self, spot, model):
        if(abs(model.texp - self.texp)>1e-12):
            print('Resetting texp of model ...')
            model.setparams(texp=self.texp)
        return model.delta(self.strike, spot, cp_sign=self.cp_sign)        

In [227]:
bsm_model = BSM_Ver3(vol=0.2, texp=1)
tc_c105_dec = OptionContract('Tencent', 'call', 105, dexp=dt.date(2017, 12, 25))
tc_p95_dec = OptionContract('Tencent', 'put', 95, dexp=dt.date(2017, 12, 25))
print(tc_c105_dec.toString())
print(tc_p95_dec.toString())

call option on Tencent struck at 105.0 maturing on 2017.12.25
put option on Tencent struck at 95.0 maturing on 2017.12.25


In [228]:
tc_spot = 100
price1 = tc_c105_dec.price(tc_spot, model=bsm_model)
price2 = tc_p95_dec.price(tc_spot, model=bsm_model)
delta = tc_c105_dec.delta(tc_spot, model=bsm_model)

print( price1, price2, delta )

Resetting texp of model ...
2.36574200253 2.17175790024 0.345833524173


In [229]:
print(bsm_model.__dict__)

{'texp': 0.2929500342231348, 'vol': 0.2, 'intr': 0.0, 'disc_fac': 1.0, 'divr': 0.0, 'vol_std': 0.10824971763900999, 'div_fac': 1.0}
